<a href="https://colab.research.google.com/github/TinsaeTadesse17/Gaussian-splatting/blob/main/unsupervised_structural_decomposition_of_3D_objects_using_Gaussian_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================
# Gaussian Clustering + Matplotlib Rendering in Colab
# ===============================

!pip install gdown plyfile scikit-learn numpy open3d matplotlib

import gdown
import numpy as np
from plyfile import PlyData
from sklearn.cluster import KMeans
import open3d as o3d
import matplotlib.pyplot as plt
import random
import os

# ========= Step 1: Download .ply from Google Drive =========
file_id = "1lld6H82jGRrfdZTH0UX0ysJqSPRFfMJ7"  # your file ID
url = f"https://drive.google.com/uc?id={file_id}"
ply_path = "/content/pinecone_gaussian.ply"
gdown.download(url, ply_path, quiet=False)
print(f"Downloaded Gaussian splat to: {ply_path}")

# ========= Step 2: Load .ply Gaussian splat =========
ply = PlyData.read(ply_path)
data = ply['vertex'].data

positions = np.stack([data['x'], data['y'], data['z']], axis=-1)

# ========= Step 3: Auto-detect color fields =========
if all(k in data.dtype.names for k in ['red','green','blue']):
    colors = np.stack([data['red'], data['green'], data['blue']], axis=-1) / 255.0
elif all(k in data.dtype.names for k in ['color_0','color_1','color_2']):
    colors = np.stack([data['color_0'], data['color_1'], data['color_2']], axis=-1)
    colors = np.clip(colors, 0, 1)
elif all(k in data.dtype.names for k in ['f_dc_0','f_dc_1','f_dc_2']):
    colors = np.stack([data['f_dc_0'], data['f_dc_1'], data['f_dc_2']], axis=-1)
    if colors.min() < 0:
        colors = 0.5 + 0.5 * colors
    colors = np.clip(colors, 0, 1)
else:
    raise ValueError("No recognizable color fields found in the PLY file.")

# ========= Step 4: Optional Gaussian-specific features =========
has_opacity = 'opacity' in data.dtype.names
has_scale = all(k in data.dtype.names for k in ['scale_0','scale_1','scale_2'])

opacity = np.expand_dims(data['opacity'], axis=-1) if has_opacity else None
scales = np.stack([data['scale_0'], data['scale_1'], data['scale_2']], axis=-1) if has_scale else None

# ========= Step 5: Build feature vector =========
feature_list = [positions * 1.0, colors * 0.5]
if opacity is not None:
    feature_list.append(opacity * 0.2)
if scales is not None:
    feature_list.append(scales * 0.3)

features = np.hstack(feature_list)

# ========= Step 6: KMeans clustering =========
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
labels = kmeans.fit_predict(features)

cluster_colors = np.array([
    [random.random(), random.random(), random.random()]
    for _ in range(num_clusters)
])
colored_points = cluster_colors[labels]

# Save clustered .ply
full_clustered_pcd = o3d.geometry.PointCloud()
full_clustered_pcd.points = o3d.utility.Vector3dVector(positions)
full_clustered_pcd.colors = o3d.utility.Vector3dVector(colored_points)
o3d.io.write_point_cloud("/content/pinecone_clustered.ply", full_clustered_pcd)
print("Clustered .ply saved to /content/pinecone_clustered.ply")

# ========= Step 7: Matplotlib-based rendering =========
def render_pointcloud_matplotlib(pcd, out_path):
    xyz = np.asarray(pcd.points)
    rgb = np.asarray(pcd.colors)
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c=rgb, s=1)
    ax.set_axis_off()
    plt.savefig(out_path)
    plt.close()

# ========= Step 8: Render overview + each cluster =========
os.makedirs("/content/cluster_renders", exist_ok=True)

render_pointcloud_matplotlib(full_clustered_pcd, "/content/cluster_renders/overview.png")

for c in range(num_clusters):
    mask = (labels == c)
    cluster_pcd = o3d.geometry.PointCloud()
    cluster_pcd.points = o3d.utility.Vector3dVector(positions[mask])
    cluster_pcd.colors = o3d.utility.Vector3dVector(np.tile(cluster_colors[c], (mask.sum(), 1)))
    render_pointcloud_matplotlib(cluster_pcd, f"/content/cluster_renders/cluster_{c}.png")

print("All renders saved to /content/cluster_renders/")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


Downloading...
From (original): https://drive.google.com/uc?id=1lld6H82jGRrfdZTH0UX0ysJqSPRFfMJ7
From (redirected): https://drive.google.com/uc?id=1lld6H82jGRrfdZTH0UX0ysJqSPRFfMJ7&confirm=t&uuid=d5ff368d-0016-4058-895e-ac43dc49f172
To: /content/pinecone_gaussian.ply
100%|██████████| 766M/766M [00:08<00:00, 87.8MB/s]


Downloaded Gaussian splat to: /content/pinecone_gaussian.ply
Clustered .ply saved to /content/pinecone_clustered.ply
All renders saved to /content/cluster_renders/


In [3]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define source folder and destination path
source_folder = '/content/cluster_renders'
destination_folder = '/content/drive/My Drive/cluster_renders'

# Copy the entire folder to Google Drive
shutil.copytree(source_folder, destination_folder)

print(f"Folder saved to: {destination_folder}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder saved to: /content/drive/My Drive/cluster_renders
